In [4]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

pd.set_option('display.float_format', lambda x: f'{x :.2f}')
pd.set_option('display.max_columns', None)
from cleaner import DataCleaner
from get_dataframes import GetDataframes
import hjson

from datetime import datetime

import matplotlib.pyplot as plt
import explo_data_analysis.eda_movies as eda

with open("config.hjson") as fp:
    config = hjson.load(fp)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
tmdb = "clean_datasets/tmdb_updated.parquet"
actors = "clean_datasets/actors_movies.parquet"
directors = "clean_datasets/directors_movies.parquet"
movies = "clean_datasets/movies_cleaned.parquet"

tmdb = pd.read_parquet(tmdb)
actors = pd.read_parquet(actors)
directors = pd.read_parquet(directors)
movies = pd.read_parquet(movies)

In [6]:
col_to_keep = [
    "titre_id",
    "titre_str",
    "titre_genres",
    "rating_avg",
    "rating_votes"
]

movies = movies[col_to_keep]

In [7]:
directors_list_id = directors["titre_id"]

condi = movies["titre_id"].isin(directors_list_id)
condi2 = actors["titre_id"].isin(directors_list_id)

movies = movies[condi]
actors = actors[condi2]

In [8]:
actors_list_id = actors["titre_id"]

condi = movies["titre_id"].isin(actors_list_id)
condi2 = directors["titre_id"].isin(actors_list_id)

movies = movies[condi]
directors = directors[condi2]

In [9]:
col_to_keep = [
    "imdb_id",
    "overview"
]

tmdb = tmdb[col_to_keep]

In [10]:
col_to_keep = [
    "titre_id",
    "person_name"
]

actors = actors[col_to_keep]

In [11]:
col_to_keep = [
    "titre_id",
    "person_name"
]

directors = directors[col_to_keep]

In [12]:
actors.loc[:, "person_name"] = actors["person_name"].str.split(", ")
directors.loc[:, "person_name"] = directors["person_name"].str.split(", ")

In [13]:
# actors
person_name = actors.groupby("titre_id")["person_name"].sum().reset_index()
person_list = person_name["person_name"].to_list()

In [14]:
# directors
directors_name = directors.groupby("titre_id")["person_name"].sum().reset_index()
directors_list = directors_name["person_name"].to_list()

In [15]:
movies["stars"] = person_list
movies["directors"] = directors_list
movies

,titre_id,titre_str,titre_genres,rating_avg,rating_votes,stars,directors
0,tt0035423,Kate & Leopold,"[Comedy, Fantasy, Romance]",6.40,87897,"[Meg Ryan, Hugh Jackman, Liev Schreiber, Breck...",[James Mangold]
1,tt0052832,The Fugitive Kind,"[Drama, Romance]",7.10,6914,"[Anna Magnani, Marlon Brando, Joanne Woodward,...",[Sidney Lumet]
2,tt0052997,The League of Gentlemen,"[Comedy, Crime, Drama]",7.20,5105,"[Jack Hawkins, Roger Livesey, Nigel Patrick, R...",[Basil Dearden]
3,tt0053459,Eyes Without a Face,"[Drama, Horror]",7.60,34278,"[Pierre Brasseur, Alexandre Rignault, Alida Va...",[Georges Franju]
5,tt0053579,Late Autumn,"[Comedy, Drama]",7.90,5854,"[Setsuko Hara, Keiji Sada, Mariko Okada, Yôko ...",[Yasujirô Ozu]
...,...,...,...,...,...,...,...
8288,tt9873892,They Cloned Tyrone,"[Comedy, Mystery, Sci-Fi]",6.60,33197,"[Kiefer Sutherland, Jamie Foxx, John Boyega, T...",[Juel Taylor]
8290,tt9883996,Dream Horse,"[Biography, Comedy, Drama]",6.90,6589,"[Alan David, Lynda Baron, Owen Teale, Toni Col...",[Euros Lyn]
8291,tt9893250,I Care a Lot,"[Comedy, Crime, Drama]",6.40,141372,"[Dianne Wiest, Rosamund Pike, Peter Dinklage, ...",[J Blakeson]
8292,tt9900782,Kaithi,"[Action, Adventure, Crime]",8.50,39546,"[Narain, Karthi, George Maryan, Arjun Das]",[Lokesh Kanagaraj]


In [16]:
merged = pd.merge(
    movies,
    tmdb,
    left_on = "titre_id",
    right_on = "imdb_id"
)

In [17]:
merged.drop(["titre_id", "imdb_id"], axis = 1, inplace = True)

In [18]:
merged

,titre_str,titre_genres,rating_avg,rating_votes,stars,directors,overview
0,Kate & Leopold,"[Comedy, Fantasy, Romance]",6.40,87897,"[Meg Ryan, Hugh Jackman, Liev Schreiber, Breck...",[James Mangold],When her scientist ex-boyfriend discovers a po...
1,The Fugitive Kind,"[Drama, Romance]",7.10,6914,"[Anna Magnani, Marlon Brando, Joanne Woodward,...",[Sidney Lumet],"Val Xavier, a drifter of obscure origins, arri..."
2,The League of Gentlemen,"[Comedy, Crime, Drama]",7.20,5105,"[Jack Hawkins, Roger Livesey, Nigel Patrick, R...",[Basil Dearden],Involuntarily-retired Colonel Hyde recruits se...
3,Eyes Without a Face,"[Drama, Horror]",7.60,34278,"[Pierre Brasseur, Alexandre Rignault, Alida Va...",[Georges Franju],Dr. Génessier is riddled with guilt after an a...
4,Late Autumn,"[Comedy, Drama]",7.90,5854,"[Setsuko Hara, Keiji Sada, Mariko Okada, Yôko ...",[Yasujirô Ozu],A woman and her daughter are each forced to co...
...,...,...,...,...,...,...,...
7411,They Cloned Tyrone,"[Comedy, Mystery, Sci-Fi]",6.60,33197,"[Kiefer Sutherland, Jamie Foxx, John Boyega, T...",[Juel Taylor],A series of eerie events thrusts an unlikely t...
7412,Dream Horse,"[Biography, Comedy, Drama]",6.90,6589,"[Alan David, Lynda Baron, Owen Teale, Toni Col...",[Euros Lyn],"The inspiring true story of Dream Alliance, an..."
7413,I Care a Lot,"[Comedy, Crime, Drama]",6.40,141372,"[Dianne Wiest, Rosamund Pike, Peter Dinklage, ...",[J Blakeson],A court-appointed legal guardian defrauds her ...
7414,Kaithi,"[Action, Adventure, Crime]",8.50,39546,"[Narain, Karthi, George Maryan, Arjun Das]",[Lokesh Kanagaraj],"Dilli, a convicted criminal, is out on parole ..."
